# Volume-Based Estimators

When TARDIS runs, we enter into a loop with two main components: a Monte Carlo iteration occurs, and then the plasma state is updated based on the "estimators" described in this page. These estimators use the Monte Carlo packets to estimate how the light-matter interactions in the supernova affect the conditions in the ejecta. This concept was originally developed by <strong data-cite="Lucy1999">[]</strong> and successively refined by <strong data-cite="Lucy1999a">[]</strong>, <strong data-cite="Lucy2002">[]</strong> and <strong data-cite="Lucy2003">[]</strong>.

## Theory

### J and nu_bar

Ordinarily, TARDIS is not concerned about the physical amount of time a packet spends traveling through the ejecta. Instead, we consider the "time of simulation" $\Delta t$ which is chosen to be the amount of time in which the photosphere emits the ensemble of packets (see [Energy Packet Initialization](../montecarlo/initialization.ipynb)). When looking at the estimators, a slightly different interpretation of the packets is necessary. Here, we view the packets as not carrying a discrete amount of energy $\varepsilon$ that is emitted in a time interval $\Delta t$, but as being a flow of energy that carries an energy $\varepsilon$ over a time $\Delta t$ -- that is, each packet is carrying a luminosity (energy per unit time) of $L = \frac{\varepsilon}{\Delta t}$. Now, we can say that if a packet spends a time $\delta t$ in the supernova's ejecta, it contributes an energy of $L\delta t= \frac{\varepsilon}{\Delta t}\delta t$ into the radiation energy of the ejecta.

To account for the effects of the Monte Carlo packets on the ejecta, TARDIS uses the packets to first determine the average radiation energy density $E$ throughout each shell, where the energy density is the total radiation energy in the shell divided by the volume of the shell $V=\frac{4}{3}\pi (r_\mathrm{outer}^3-r_\mathrm{inner}^3)$. Therefore, we add up the amount of energy each packet contributes to the radiation energy in that shell, and divide by the total volume of the shell:
$$E=\frac{1}{V}\sum_i L_i\delta t_i=\frac{1}{V}\sum_i \frac{\varepsilon_i}{\Delta t}\delta t_i = \frac{1}{V\Delta t}\sum_i \varepsilon_i\delta t_i$$
where we sum over every Monte Carlo packet in the shell. Note that we are interested in the energy density in the co-moving frame (i.e. the energy density "according to the plasma," see [reference frames](../montecarlo/propagation.rst#reference-frames)). Now, we note that the amount of time the Monte Carlo packet spends in a shell is $\delta t = \frac{l_i}{c}$ where $l$ is the distance that the packet travels through the shell. Thus, our estimator is
$$E=\frac{1}{V\Delta t}\sum_i \varepsilon_i\frac{l_i}{c} = \frac{1}{cV\Delta t}\sum_i \varepsilon_i l_i.$$

Using this energy density, we can then calculate the mean radiation intensity $J$ in that shell using the relation $J=\frac{c}{4\pi} E$, which gives us
$$J=\frac{1}{4\pi V\Delta t}\sum_i \varepsilon_i l_i.$$
Since along any path the co-moving energy of the packet is continuously doppler shifted, we approximate this estimator using the co-moving energy at the beginning of the packet's path (theoretically, the midpoint of the path would be a better option. However, we use the beginning of the path for computational ease at a very small cost to the estimator's accuracy).

Next, we calculate the mean radiation frequency in each shell. For this, in each shell we add up the frequency of each packet weighted by the intensity they contribute to the shell. Remembering that intensity is $\frac{c}{4\pi}$ times the energy density, and as before each packet contributes an energy of $\frac{\varepsilon_i l_i}{c\Delta t}$ and thus energy density of $\frac{\varepsilon_i l_i}{cV\Delta t}$ to its shell, we have that each packet contributes an intensity of $\frac{\varepsilon_i l_i}{4\pi V\Delta t}$ to its shell. So,
$$\bar \nu = \sum_i \frac{\varepsilon_i l_i}{4\pi V \Delta t}  \nu_i = \frac{1}{4\pi V \Delta t}\sum_i \varepsilon_i \nu_i l_i$$
where once again the co-moving energy and frequency of each packet are taken at the beginning of the packet's path.

<div class="alert alert-info">
    
Note

Both estimators take on a different value in each shell.

</div>

These estimators allow us to calculate the [radiative temperature](../setup/model.ipynb#Radiative-Temperature) $T_\mathrm{rad}$ and [dilution factor](../setup/model.ipynb#Dilution-Factor) $W$ in each shell using

$$T_{\mathrm{rad}} = \frac{h}{k_{\mathrm{B}}} \frac{\pi^4}{360 \zeta(5)} \frac{\bar \nu}{J}$$

and

$$W = \frac{\pi J}{\sigma_{\mathrm{R}} T_{\mathrm{rad}}^4}$$

where $h$ is Planck's constant, $k_{\mathrm{B}}$ is Boltzmann's constant, $\sigma_{\mathrm{R}}$ is the Stefan–Boltzmann constant, and $\zeta$ is the Riemann zeta function. The equation for $W$ comes from the fact that the dilution factor is the ratio of the actual mean intensity to that of a blackbody, which is $J_{\mathrm{blackbody}}=\frac{\sigma_{\mathrm{R}} T^4}{\pi}$.

The new $T_\mathrm{rad}$ and $W$ are then used as inputs to the updated [plasma calculations](../setup/plasma/index.rst) which account for the effect of the Monte Carlo packets on the plasma state (precisely, these calculated $T_\mathrm{rad}$ and $W$ help determine the $T_\mathrm{rad}$ and $W$ used as inputs in the plasma calculations -- see [convergence](convergence.rst#t-rad-and-w) for specifics).


### J_blue

Another estimator, called the ``J_blue`` or $J^b_{lu}$ estimator, is unlike the two previous estimators discussed. Instead of storing the mean intensity over the entire spectrum, it stores the intensity at a specific frequency. More precisely, since frequency is a continuum, it stores the intensity per unit frequency. In each shell, we record the intensity per unit frequency at the blue end (higher frequency end; this is where the "$b$" superscript in $J^b_{lu}$ comes from) of each line transition -- that is, if a line transition $l\rightarrow u$ (from the lower energy level $l$ to the upper energy level $u$, hence the $lu$ in $J^b_{lu}$) has a frequency $\nu_{lu}$, the mean intensity between $\nu_{lu}$ and $\nu_{lu}+d\nu$ is $J^b_{lu}d\nu$. **This means that the** $J^b_{lu}$ **estimator has an entry for each atomic line in each shell.** Now, using our previous $J$ estimator, we have
$$J^b_{lu}d\nu = \frac{1}{4\pi V\Delta t}\sum_i \varepsilon_i dl_i$$
where $dl_i$ is the infinitesimal distance that the packet travels while it has a co-moving frequency between $\nu_{lu}$ and $\nu_{lu}+d\nu$.

Now, say the packet with lab frequency $\nu_\mathrm{lab}$ has a co-moving frequency of $\nu_{lu}$ at a radius $r_1$ and propagation direction $\mu_1$, and it has a co-moving frequency of $\nu_{lu}+d\nu$ at a radius $r_2$ and propagation direction $\mu_2$. Then (see [reference frames](../montecarlo/propagation.rst#reference-frames)):
$$\nu_{lu}=\left(1-\frac{r_1\mu_1}{ct_\mathrm{explosion}}\right)\nu_\mathrm{lab}$$
and
$$\nu_{lu}+d\nu=\left(1-\frac{r_2\mu_2}{ct_\mathrm{explosion}}\right)\nu_\mathrm{lab}.$$
But then subtracting, we get
$$d\nu = (r_2\mu_2-r_1\mu_1)\frac{\nu_\mathrm{lab}}{ct_\mathrm{explosion}}=dl*\frac{\nu_\mathrm{lab}}{ct_\mathrm{explosion}}$$
(for the last equality, see [propagation in a spherical domain](../montecarlo/propagation.rst#propagation-in-a-spherical-domain)).

But now inputting this into the equation for $J^b_{lu}$ (using $\frac{dl_i}{d\nu}=\frac{ct_\mathrm{explosion}}{\nu_\mathrm{lab,i}}$), we get
$$J^b_{lu} = \frac{ct_\mathrm{explosion}}{4\pi V\Delta t}\sum_i \frac{\varepsilon_i}{\nu_\mathrm{lab,i}}.$$


## Implementation

As previously discussed, a major component of each Monte Carlo iteration is the packet propagation process. During the packet propagation process this step, the $J$ and $\bar \nu$ estimators are updates every time a packet is moved to the next event location. Specifically, every time a packet is moved, $\varepsilon l$ is added to the "running total" $J$ estimator in the shell where the packet is, and $\varepsilon \nu l$ is added to the "running total" $\bar\nu$ estimator in the shell where the packet is (where $l$ is the distance the packet is moved, and $\varepsilon$ and $\nu$ are respectively the packet's co-moving energy and frequency at the beginning of the packet's path). The factor of $\frac{1}{4\pi V\Delta t}$, for computational ease, is not attached to the estimators but is included during the calculation of $T_\mathrm{rad}$ and $W$ from the estimators. Specifically, we use
$$T_\mathrm{rad}=\frac{h}{k_{\mathrm{B}}} \frac{\pi^4}{360 \zeta(5)} \frac{\sum_i \varepsilon_i \nu_i l_i}{\sum_i \varepsilon_i l_i} = \frac{h}{k_{\mathrm{B}}} \frac{\pi^4}{360 \zeta(5)} \frac{\mathrm{real\ nu\_ bar\ estimator}}{\mathrm{real\ J\ estimator}}$$
and
$$W = \frac{\sum_i \varepsilon_i l_i}{4\sigma_{\mathrm{R}} T_{\mathrm{rad}}^4V\Delta t} = \frac{\mathrm{real\ J\ estimator}}{4\sigma_{\mathrm{R}} T_{\mathrm{rad}}^4V\Delta t}$$

After each Monte Carlo iteration, the `advance_state()` method is called on the `Simulation` object. The estimators are then used to update the radiative temperature and dilution factor according to the [convergence strategy](convergence.rst), and the plasma state is recalculated (see [plasma calculations](../setup/plasma/index.rst)) using the updated radiative temperature and dilution factor as inputs. This process repeats until the final iteration or until convergence has been reached (see [convergence](convergence.rst#Convergence-Criteria)).

Similarly, during the propagation process, every time a packet passes through a Sobolev point, meaning it comes in resonance with an atomic line (and thus reaches the frequency targeted by $J^b_{lu}$ -- not necessarily going through a line interaction), the $J^b_{lu}$ for that atomic transition in the shell it is in is incremented by $\frac{\varepsilon}{\nu_\mathrm{lab}}$, where $\varepsilon$ is the packet's energy qnd $\nu_\mathrm{lab}$ is the packet's lab-frame frequency. As before, for computational ease, the factor $\frac{ct_\mathrm{explosion}}{4\pi V \Delta t}$ is included in any calculations using the estimator.

<div class="alert alert-info">
    
Note

Since the ``J_blue`` estimator is updated every time a packet comes into resonance with an atomic line (not necessarily going through a line interaction), the estimator is only equal to zero in some shell for a specific line if no packet comes into resonance with that line within the shell.

</div>

If set to detailed mode (see [plasma configuration](../../io/configuration/components/plasma.rst)), the `J_blue` plasma property will will be replaced with the value of the $J^b_{lu}$ estimator (the raw estimator times the factor of $\frac{ct_\mathrm{explosion}}{4\pi V \Delta t}$). Otherwise, the `J_blue` in the plasma are calculated as they typically are in the plasma calculations, and the $J^b_{lu}$ estimator is only used for the [formal integral](../spectrum/sourceintegration.rst).

## Code Example

We now show a detailed example of how the plasma is updated using the estimators after a Monte Carlo iteration. First, we import the needed packages and set up a simulation (see [Setting Up the Simulation](../setup/index.rst)):

In [1]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.io.atom_data.util import download_atom_data
import numpy as np

# We download the atomic data needed to run this notebook
download_atom_data('kurucz_cd23_chianti_H_He')

/usr/share/miniconda3/envs/tardis/lib/python3.8/site-packages/traitlets/traitlets.py:3044: FutureWarning: --rc={'figure.dpi': 96} for dict-traits is deprecated in traitlets 5.0. You can pass --rc <key=value> ... multiple times to add items to a dict.
  warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)

model = sim.model
plasma = sim.plasma
runner = sim.runner

/usr/share/miniconda3/envs/tardis/lib/python3.8/site-packages/tardis-22.4.24.1.dev950+gd3d0dacc-py3.8.egg/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning: invalid value encountered in true_divide
  (g_lower * n_upper) / (g_upper * n_lower)


/usr/share/miniconda3/envs/tardis/lib/python3.8/site-packages/tardis-22.4.24.1.dev950+gd3d0dacc-py3.8.egg/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning: invalid value encountered in true_divide
  (g_lower * n_upper) / (g_upper * n_lower)


We show the initial radiative temperature, dilution factor, electron densities, and tau sobolevs in each shell:

In [3]:
model.t_rad

<Quantity [9926.50196546, 9911.63537753, 9896.81325339, 9882.03539385,
           9867.30160093, 9852.6116778 , 9837.96542884, 9823.36265956,
           9808.80317663, 9794.28678787, 9779.81330223, 9765.3825298 ,
           9750.99428177, 9736.64837045, 9722.34460927, 9708.08281272,
           9693.8627964 , 9679.68437699, 9665.54737223, 9651.45160094] K>

In [4]:
model.w

array([0.40039164, 0.33245223, 0.28966798, 0.2577141 , 0.23224568,
       0.21120466, 0.19341188, 0.17811402, 0.16479446, 0.1530809 ,
       0.14269498, 0.13342262, 0.12509541, 0.11757849, 0.11076215,
       0.10455605, 0.09888494, 0.09368554, 0.08890421, 0.08449514])

In [5]:
plasma.electron_densities

0     2.865134e+09
1     2.182365e+09
2     1.678840e+09
3     1.303472e+09
4     1.020811e+09
5     8.059447e+08
6     6.411609e+08
7     5.137297e+08
8     4.144079e+08
9     3.364195e+08
10    2.747519e+08
11    2.256656e+08
12    1.863476e+08
13    1.546660e+08
14    1.289928e+08
15    1.080764e+08
16    9.094799e+07
17    7.685317e+07
18    6.520063e+07
19    5.552442e+07
dtype: float64

In [6]:
plasma.tau_sobolevs

0   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 6.110053e-12   
                         177                191                 4.581877e-12   
                         178                191                 3.048177e-13   
                         138                144                 2.205327e-10   
                         177                190                 8.129552e-14   

                                                                          1   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 2.765775e-12   
                         177                191                 2.074031e-12   
                         178                191                 1.379787e-13   
                         138                144                 9.983538e-11   
                         177                190                 3.679921e-14   

                                                                          2   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 1.290338e-12   
                         177                191                 9.676133e-13   
                         178                191                 6.437225e-14   
                         138                144                 4.658121e-11   
                         177                190                 1.716821e-14   

                                                                          3   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 6.191522e-13   
                         177                191                 4.642969e-13   
                         178                191                 3.088820e-14   
                         138                144                 2.23534

We set the number of packets and we run one iteration of the Monte Carlo simulation:

In [7]:
N_packets = 10000

# Using the commented out code below, we can also get the number of packets
# from the configuration -- try it out:
#N_packets = tardis_config.no_of_packets

sim.iterate(N_packets)

TqdmHBox(children=(HTML(value='Iterations:', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

TqdmHBox(children=(HTML(value='Packets:   ', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

We now show the values of the three estimators previously mentioned (note that these are the raw estimators, and the factors of $\frac{1}{4\pi V \Delta t}$ etc are not included):

In [8]:
runner.j_estimator

array([1.13264880e+14, 9.80619708e+13, 8.77127720e+13, 7.93178023e+13,
       7.33452674e+13, 6.69288783e+13, 6.42825236e+13, 6.09589567e+13,
       5.76244911e+13, 5.49744149e+13, 5.27742170e+13, 5.11516006e+13,
       4.94278177e+13, 4.78690453e+13, 4.66549201e+13, 4.55199706e+13,
       4.46151244e+13, 4.38293494e+13, 4.31417403e+13, 4.22137375e+13])

In [9]:
runner.nu_bar_estimator

array([9.23669721e+28, 8.00440961e+28, 7.16701149e+28, 6.52233231e+28,
       6.04286574e+28, 5.48759864e+28, 5.29893354e+28, 5.02177670e+28,
       4.72808091e+28, 4.50654639e+28, 4.30918375e+28, 4.12731776e+28,
       3.97685768e+28, 3.81488727e+28, 3.68151763e+28, 3.55929590e+28,
       3.46700820e+28, 3.38472442e+28, 3.32955181e+28, 3.24241018e+28])

In [10]:
# Because most rows of the j_blue estimatior are partially or mostly
# zero, we show just rows with all nonzero elements
runner.j_blue_estimator[np.all(runner.j_blue_estimator != 0, axis=1)]

array([[2.54208693e-19, 3.98921452e-19, 3.51022121e-19, ...,
        4.78256257e-20, 1.83610133e-19, 4.62693098e-20],
       [5.25249079e-20, 3.47850697e-19, 3.53175242e-19, ...,
        4.68369084e-20, 4.76896351e-20, 1.42425885e-19],
       [5.25205919e-20, 3.47822114e-19, 3.03857848e-19, ...,
        4.68330598e-20, 4.76857164e-20, 1.42414182e-19],
       ...,
       [9.16402388e-19, 4.44096419e-19, 8.75723476e-19, ...,
        1.23926509e-18, 8.25398566e-19, 8.28841475e-19],
       [9.16135899e-19, 4.43967276e-19, 4.26276968e-19, ...,
        8.28490409e-19, 1.23557284e-18, 8.28600449e-19],
       [2.29780963e-18, 9.13630461e-19, 2.30523331e-18, ...,
        4.23437186e-19, 4.28206752e-19, 4.22703828e-19]])

We note that the shape of the j_blue estimator and the tau_sobolevs are the same: namely, each contain a value for each possible atomic line transition in each radial cell (as opposed to the other two estimators which just have one value for each cell):

In [11]:
plasma.tau_sobolevs.shape, runner.j_blue_estimator.shape

((29224, 20), (29224, 20))

We now advance the state of the simulation based on the estimators, and demonstrate this by showing the four quantities we showed before running the simulation. Compare them with their values above!

In [12]:
# When advance_state is called, a brief summary of the updated t_rad's
# and w's is given
sim.advance_state();

Shell No.,t_rad,next_t_rad,w,next_w
0,9.93e+03,1.02e+04,0.4,0.482
5,9.85e+03,1.03e+04,0.211,0.193
10,9.78e+03,1.02e+04,0.143,0.114
15,9.71e+03,9.79e+03,0.105,0.0893


/usr/share/miniconda3/envs/tardis/lib/python3.8/site-packages/tardis-22.4.24.1.dev950+gd3d0dacc-py3.8.egg/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning: invalid value encountered in true_divide
  (g_lower * n_upper) / (g_upper * n_lower)


In [13]:
model.t_rad

<Quantity [10212.74995587, 10222.33124932, 10232.84808982, 10298.01481758,
           10317.91893208, 10268.09920614, 10323.25883794, 10316.70789091,
           10275.40760376, 10266.07641642, 10225.73367215, 10104.851613  ,
           10076.03983165,  9980.40591063,  9882.13306024,  9792.27010212,
            9731.81900046,  9671.18192097,  9665.16737459,  9619.12152881] K>

In [14]:
model.w

array([0.48154582, 0.3839523 , 0.31710814, 0.25991916, 0.22230488,
       0.19324106, 0.17011455, 0.15176297, 0.13706675, 0.12362115,
       0.11375669, 0.10928659, 0.10111279, 0.09644048, 0.09283222,
       0.0893001 , 0.08539126, 0.08195894, 0.0771517 , 0.07348597])

In [15]:
plasma.electron_densities

0     2.890015e+09
1     2.199909e+09
2     1.691446e+09
3     1.313845e+09
4     1.028790e+09
5     8.114482e+08
6     6.459942e+08
7     5.175646e+08
8     4.172918e+08
9     3.387728e+08
10    2.765659e+08
11    2.267949e+08
12    1.872454e+08
13    1.552250e+08
14    1.292984e+08
15    1.082114e+08
16    9.099949e+07
17    7.684337e+07
18    6.520026e+07
19    5.549692e+07
dtype: float64

In [16]:
plasma.tau_sobolevs

0   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 3.751288e-12   
                         177                191                 2.813059e-12   
                         178                191                 1.871440e-13   
                         138                144                 1.351661e-10   
                         177                190                 4.991160e-14   

                                                                          1   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 1.623835e-12   
                         177                191                 1.217700e-12   
                         178                191                 8.100975e-14   
                         138                144                 5.850663e-11   
                         177                190                 2.160543e-14   

                                                                          2   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 7.234954e-13   
                         177                191                 5.425431e-13   
                         178                191                 3.609368e-14   
                         138                144                 2.606589e-11   
                         177                190                 9.626242e-15   

                                                                          3   \
atomic_number ion_number level_number_lower level_number_upper                 
14            5          0                  36                  0.000000e+00   
                                            35                  0.000000e+00   
                                            34                  0.000000e+00   
                         1                  35                  0.000000e+00   
                         0                  33                  0.000000e+00   
...                                                                      ...   
20            0          176                191                 3.026676e-13   
                         177                191                 2.269679e-13   
                         178                191                 1.509946e-14   
                         138                144                 1.09003